In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load
# 
# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# 
# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# 
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
# 
# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#  Tuning parameters for logistic regression model

In [2]:
import numpy as np
import random

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

## Load data

In [1]:
import pandas as pd

data_train = pd.read_csv('../../data/interim/trainFinal.csv')
data_test = pd.read_csv('../../data/interim/testFinal.csv')
data_train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity,Activity_code
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,STANDING,2
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,STANDING,2
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,STANDING,2
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,STANDING,2
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,STANDING,2


## 2. Create X and y variables

In [4]:
X_train = data_train.drop(['Activity', 'Activity_code'], axis=1)
y_train = np.array(data_train['Activity_code'])

X_test = data_test.drop(['Activity', 'Activity_code'], axis=1),
y_test = data_test['Activity_code']

## 3. Grid search

In [5]:
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Khởi tạo mô hình
model = LogisticRegression()

# Lưới siêu tham số
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],  # Loại regularization
    'C': [0.01, 0.1, 1, 10, 20],             # Lực regularization (C = 1/λ)
    'solver': ['liblinear', 'saga'],             # Bộ giải (phù hợp với penalty được chọn)
    'max_iter': [500, 1000, 2000, 3000],           # Số lần lặp tối đa
}

# Khởi tạo GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3,                    # Số lần cross-validation
    scoring='accuracy',      # Metric đánh giá
    verbose=1,               # Hiển thị chi tiết hơn tiến trình
    n_jobs=-1                # Sử dụng toàn bộ CPU
)

# Theo dõi thời gian
start_time = time.time()

# Thực hiện tìm kiếm
print("Starting GridSearchCV...")
grid_search.fit(X_train, y_train)

# Tính toán thời gian
elapsed_time = time.time() - start_time

# Kết quả
print("GridSearchCV completed!")
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print('Best params: ', grid_search.best_params_)
print('Best score: ', grid_search.best_score_)


Starting GridSearchCV...
Fitting 3 folds for each of 120 candidates, totalling 360 fits


D:\App\anaconda3\envs\ML_env\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
120 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "D:\App\anaconda3\envs\ML_env\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\App\anaconda3\envs\ML_env\lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "D:\App\anaconda3\envs\ML_env\lib\site-packages\sklearn\linear_model\_logistic.py", line 1194, in fit
    solver = _check_solver(self.solver, self.p

GridSearchCV completed!
Elapsed time: 1179.34 seconds
Best params:  {'C': 20, 'max_iter': 500, 'penalty': 'l2', 'solver': 'liblinear'}
Best score:  0.9453232749648208


## 4. Save the best model

In [16]:
import os
import json

# Dữ liệu để lưu vào file JSON
data = {
    "C": grid_search.best_params_['C'],
    "max_iter": grid_search.best_params_['max_iter'],
    "penalty": grid_search.best_params_['penalty'],
    "solver": grid_search.best_params_['solver']
}

# Đường dẫn và tên file JSON
directory = "Best_Hyperparameter/"
filename = "best_parameter_LogisticRegression.json"

# Tạo thư mục nếu chưa tồn tại
os.makedirs(directory, exist_ok=True)

# Đường dẫn đầy đủ của file JSON
file_path = os.path.join(directory, filename)

# Ghi dữ liệu vào file JSON
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, indent=4, ensure_ascii=False)

print(f"File JSON đã được tạo tại: {file_path}")


File JSON đã được tạo tại: Best_Hyperparameter/best_parameter_LogisticRegression.json


## 5. Results

##### Logistic Regression
* GridSearchCV completed!
* Elapsed time: 8571.93 seconds
* Best params:  {'C': 20, 'max_iter': 500, 'penalty': 'l2', 'solver': 'liblinear'}
* Best score:  0.945187276052812